In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
from torch.utils.data import DataLoader, Dataset, random_split
import torch
import torch.nn as nn
from torchvision.io import read_image
import torchvision.transforms as transforms
from torchvision import datasets
from torch import optim
import os

In [4]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [5]:
transforms = transforms.Compose([
    transforms.ToTensor()
])

In [27]:
class CustomDataset(Dataset):
    def __init__(self, labels_file, imgs_dir, transform=None, target_transform=None):
        self.labels = pd.read_csv(labels_file)
        self.imgs_dir = imgs_dir
        self.transform = transform
        self.target_transform = target_transform
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        #img_path = os.path.join(self.imgs_dir, self.labels.iloc[idx, 0])
        #image = torch.tensor(read_image(img_path), dtype=torch.float32)
        #label = torch.tensor(self.labels.iloc[idx, 1], dtype=torch.int64)
        #if self.transform:
        #    image = self.transform(image)
        #if self.target_transform:
        #    label = self.target_transform(label)
        #return image, label
        return self.labels

In [28]:
CustomDataset('data/Train/Train.csv', 'data/Train/', transforms).__getitem__(0)

,Unnamed: 0,Name,label
0,0,lwxcgsnhctd.jpg,1
1,1,sppvttqetkt.jpg,1
2,2,adidas_188_2024-03-21 00-21-15.jpg,0
3,3,157.jpg,0
4,4,97.jpg,0
...,...,...,...
1564,1564,yhfhmvftzfu.jpg,1
1565,1565,adidas_121_2024-03-21 00-21-14.jpg,0
1566,1566,adidas_52_2024-03-21 00-21-17.jpg,0
1567,1567,bhvlfdgfsql.jpg,1


In [15]:
df = pd.read_csv('data/Train/Train.csv')
df.shape

(1569, 2)